In [1]:
import sys, os
import numpy as np
import pandas as pd
import geopandas as gpd
sys.path.insert(0, r'Y:\champ\util\pythonlib-migration\master_versions\misc_utils')
from df_utils import df_to_excel

In [2]:
SURVEY_2023 = {'household':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20240329\BATS Unweighted Dataset\hh.csv'},
               'person':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20240329\BATS Unweighted Dataset\person.csv'},
               'day':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20240329\BATS Unweighted Dataset\day.csv'},
               'trip':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20240329\BATS Unweighted Dataset\trip.csv'},
               'vehicle':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20240329\BATS Unweighted Dataset\vehicle.csv'},
               'location':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20240329\BATS Unweighted Dataset\location.csv'},
               }


OUTDIR = r'..\..\..\..\Review_20240329'
COUNTIES = r'Q:\GIS\Boundaries\Counties\Counties.shp'

In [3]:
writer = pd.ExcelWriter(os.path.join(OUTDIR,'completeness_tables_lenient.xlsx'))

In [4]:
nine_to_county = {1:'San Francisco',
                  2:'San Mateo',
                  3:'Santa Clara',
                  4:'Alameda',
                  5:'Contra Costa',
                  6:'Solano',
                  7:'Napa',
                  8:'Sonoma',
                  9:'Marin'}

purp_num_to_name18 = {1: 'Home',
                      2: 'Work',
                      3: 'Work-related',
                      4: 'School',
                      5: 'Escort',
                      6: 'Shop',
                      7: 'Meal', 
                      8: 'Social/rec',
                      9: 'Errand/appt',
                      10: 'Change mode',
                      11: 'Spent night other home',
                      12: 'Other/missing',
                      14: 'School-related'}

purp_num_to_name23 = {1: 'Home',
                      2: 'Work',
                      3: 'Work-related',
                      4: 'School',
                      5: 'School-related',
                      6: 'Escort',
                      7: 'Shop',
                      8: 'Meal', 
                      9: 'Social/rec',
                      10: 'Errand/appt',
                      11: 'Change mode',
                      12: 'Spent night other home',
                      13: 'Other/missing',
                     }
mode_num_to_name23 = {1:'Walk (or jog/wheelchair)',
                        2:'Standard bicycle (my household\'s)',
                        3:'Borrowed bicycle (e.g., a friend\'s)',
                        4:'Other rented bicycle',
                        5:'Other',
                        6:'Household vehicle 1',
                        7:'Household vehicle 2',
                        8:'Household vehicle 3',
                        9:'Household vehicle 4',
                        10:'Household vehicle 5',
                        11:'Household vehicle 6',
                        12:'Household vehicle 7',
                        13:'Household vehicle 8',
                        14:'Household vehicle 9',
                        15:'Household vehicle 10',
                        16:'Other vehicle in household',
                        17:'Rental car',
                        18:'Carshare service (e.g., Zipcar)',
                        21:'Vanpool',
                        22:'Other vehicle (not my household\'s)',
                        23:'Local (public) bus',
                        24:'School bus',
                        25:'Intercity bus (e.g., Greyhound, Megabus)',
                        26:'Other private shuttle/bus (e.g., a hotel\'s, an airport\'s)',
                        27:'Paratransit/Dial-A-Ride',
                        28:'Other bus',
                        30:'BART',
                        31:'Airplane/helicopter',
                        33:'Car from work',
                        34:'Friend/relative/colleague\'s car',
                        36:'Regular taxi (e.g., Yellow Cab)',
                        38:'University/college shuttle/bus',
                        41:'Intercity/Commuter rail (e.g., Altamount ACE, Amtrak, Caltrain)',
                        42:'Other rail',
                        43:'Skateboard or rollerblade',
                        44:'Golf cart',
                        45:'ATV',
                        47:'Other motorcycle in household',
                        49:'Uber, Lyft, or other smartphone-app ride service',
                        53:'MUNI Metro',
                        54:'Other motorcycle (not my household\'s)',
                        55:'Express bus or Transbay bus',
                        59:'Peer-to-peer car rental (e.g., Turo)',
                        60:'Other hired car service (e.g., black car, limo)',
                        61:'Rapid transit bus (BRT)',
                        62:'Employer-provided shuttle/bus',
                        63:'Medical transportation service',
                        67:'Local (private) bus (e.g., RapidShuttle, SuperShuttle)',
                        68:'Cable car or streetcar',
                        69:'Bike-share - standard bicycle',
                        70:'Bike-share - electric bicycle',
                        73:'Moped-share (e.g., Scoot)',
                        74:'Segway',
                        75:'Other',
                        76:'Carpool match (e.g., Waze Carpool)',
                        77:'Personal scooter or moped (not shared)',
                        78:'Public ferry or water taxi',
                        80:'Other boat (e.g., kayak)',
                        82:'Electric bicycle (my household\'s)',
                        83:'Scooter-share (e.g., Bird, Lime)',
                        100:'Household vehicle (or motorcycle)',
                        101:'Other vehicle (e.g., friend\'s car, rental, carshare, work car)',
                        102:'Bus, shuttle, or vanpool',
                        103:'Bicycle',
                        104:'Other',
                        105:'Rail (e.g., train, light rail, trolley, BART, MUNI Metro)',
                        106:'Uber/Lyft, taxi, or car service',
                        107:'Micromobility (e.g., scooter, moped, skateboard)',
                        995:'Missing Response',
                   }
county_order = ['San Francisco','San Mateo','Santa Clara','Alameda','Contra Costa','Solano','Napa','Sonoma','Marin']

In [5]:
class Survey(object):
    def __init__(self, household, person, day, trip, vehicle, location):
        self.hh = pd.read_csv(**household)
        self.person = pd.read_csv(**person)
        self.day = pd.read_csv(**day)
        self.trip = pd.read_csv(**trip)
        self.vehicle = pd.read_csv(**vehicle)
        self.location = pd.read_csv(**location)
        
        self.trip['purpose'] = self.trip['d_purpose_category']
        self.trip.loc[self.trip['d_purpose_category'].eq(1), 'purpose'] = self.trip['o_purpose_category']
        # household counts
        ## trips
        tc = self.trip.groupby('hh_id', as_index=False).size().rename(columns={'size':'trips'})
        tc = pd.merge(self.hh[['hh_id']], tc, on='hh_id', how='left')
        tc['trips'] = tc['trips'].fillna(0)
        
        ## days
        dc = self.day.groupby('hh_id', as_index=False).size().rename(columns={'size':'days'})
        dc = pd.merge(self.hh[['hh_id']], dc, on='hh_id', how='left')
        dc['days'] = dc['days'].fillna(0)
        
        ## persons
        pc = self.person.groupby('hh_id', as_index=False).size().rename(columns={'size':'persons'})
        pc = pd.merge(self.hh[['hh_id']], pc, on='hh_id', how='left')
        pc['persons'] = pc['persons'].fillna(0)
        
        self.hh_counts = pd.merge(pc, pd.merge(dc, tc, on='hh_id'), on='hh_id')
        
        # person counts
        ## trips
        tc = self.trip.groupby('person_id', as_index=False).size().rename(columns={'size':'trips'})
        tc = pd.merge(self.person[['person_id']], tc, on='person_id', how='left')
        tc['trips'] = tc['trips'].fillna(0)
        
        ## days
        dc = self.day.groupby('person_id', as_index=False).size().rename(columns={'size':'days'})
        dc = pd.merge(self.person[['person_id']], dc, on='person_id', how='left')
        dc['days'] = dc['days'].fillna(0)
        self.person_counts = pd.merge(tc, dc, on='person_id')
  
        # person day counts
        ## trips
        if 'day_id' not in self.trip.columns:
            self.trip['day_id'] = self.trip.apply(lambda x: '{}{:02d}'.format(x['person_id'], x['day_num']), axis=1)
        if 'day_id' not in self.day.columns:
            self.day['day_id'] = self.day.apply(lambda x: '{}{:02d}'.format(x['person_id'], x['day_num']), axis=1)
        tc = self.trip.groupby('day_id', as_index=False).size().rename(columns={'size':'trips'})
        tc = pd.merge(self.day[['day_id']], tc, on='day_id', how='left')
        tc['trips'] = tc['trips'].fillna(0)
        self.day_counts = tc.copy()
        
        # trip location counts
        tl = self.location.drop_duplicates().groupby('trip_id', as_index=False).size().rename(columns={'size':'locations'})
        tl = pd.merge(self.trip, tl, on='trip_id', how='left').fillna(0)
        self.trip_locations = tl.copy()

In [6]:
s23 = Survey(**SURVEY_2023)

# SFCTA Completeness Evaluation
## Trip Completeness
add sfcta "trip complete" flag, cross-tab against rsg completeness

In [7]:
#for df, s in [(s23.hh, 'hh'), (s23.person, 'person'), (s23.day, 'day'), (s23.trip, 'trip')]:
#    x = pd.DataFrame((df.map(lambda x: x==995 or pd.isnull(x)) * 1).sum(), columns=['missing'])
#    x['pct_missing'] = x/len(df)
#    x.loc['n'] = len(df)
#    x.to_csv(r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\Review_20240329\{}_complete.csv'.format(s))

In [8]:
trip = pd.merge(s23.person[['person_id','diary_platform']], s23.trip)

### Check the values of system-generated values to ensure they are reported and values are logical

In [9]:
# check arrive date complete
pd.to_datetime(trip['arrive_date']).agg(['min','max'])

min   2023-05-07
max   2024-01-19
Name: arrive_date, dtype: datetime64[ns]

In [10]:
# check dpart date complete
pd.to_datetime(trip['depart_date']).agg(['min','max'])

min   2023-05-07
max   2024-01-19
Name: depart_date, dtype: datetime64[ns]

In [11]:
trip[['hh_id','person_id','person_num','hh_is_complete','day_id','day_num','travel_date',
      'travel_dow','trip_id','trip_num','day_is_complete','trip_survey_complete','copied_from_proxy',
      'days_first_trip','days_last_trip',
      'depart_hour','depart_minute','depart_seconds','arrive_hour','arrive_minute','arrive_second',
      'o_lon','o_lat','d_lon','d_lat']].agg(['min','max']).T

,min,max
hh_id,23000075,23803687
person_id,2300007501,2380368701
person_num,1,9
hh_is_complete,1,1
day_id,230000750101,238036870107
day_num,1,8
travel_date,2023-05-07,2024-01-19
travel_dow,1,7
trip_id,2300007501001,2380368701042
trip_num,1,230


### Driving Trips
1. `missing_trip_mode_flag`: missing user-reporte mode info in `mode_1`, `mode_2`, `mode_3`, or `mode_4`
2. `missing_trip_purpose_flag`: missing purpose info in `o_purpose`, `o_purpose_category`, `d_purpose`, or `d_purpose_category
3. `missing_trip_driver_flag`: missing driver info, for drive trips with 2 or more driving age person
4. `missing_trip_transit_accegr`: missing `transit_access` or `transit_egress` for any trip with a transit component
5. `missing_trip_any_flag`: missing any of the above

In [12]:
## Create the trip flags

# car modes where the survey respondent could conceivably be driving (excl shuttle, tnc, taxi, micromobility, atv, golf cart)
flag = pd.Series(index=trip.index, data=False)
for c in ['mode_1','mode_2','mode_3','mode_4']:
    flag = flag | trip[c].isin([6,7,8,9,10,11,12,13,14,15,16,17,18,22,33,34,47,54,76,100,101])
    
for c in ['mode_type']:
    flag = flag | trip[c].isin([8,9])
    
# 9 and 10 are "dropped off".  Possibly remove these.
for c in ['transit_access', 'transit_egress']:
    flag = flag | trip[c].isin([7,8,9,10])
    
drive_flag = flag
    
# identify parties with more than one driving-aged person
driving_age = s23.person.pivot(index='hh_id', columns='person_num', values='age').gt(3).rename(columns={c: 'hh_member_{}_da'.format(c) for c in np.arange(1,9)})
trip['person_num'] = trip['person_id'].map(lambda x: int(str(x)[-2:]))
trip = pd.merge(trip,driving_age, on='hh_id',how='left')

flag = False
subflag = False
for i in np.arange(1,9):
    # now check whether there's a driving-age hh member
    flag = flag | (trip['person_num'].ne(i) & trip['hh_member_{}'.format(i)].eq(1) & trip['hh_member_{}_da'.format(i)].eq(True))
    
    # if this person is not driver age, don't check
    subflag = subflag | (trip['person_num'].eq(i) & ~trip['hh_member_{}_da'.format(i)])
    
# check if there are non-hh party members (might be driving age)
flag = flag | trip['num_non_hh_travelers'].gt(0)

# contains some else who might be a driver, and this person can also be a driver
multi_da_flag = flag & ~subflag

# flag if the trip includes any drive mode, party size > 1, and there's no driver response
missing_trip_driver_flag = drive_flag & multi_da_flag & (trip['num_travelers'].gt(1) | pd.isnull(trip['num_travelers'])) & ((trip['driver'].eq(995)) | pd.isnull(trip['driver']))

# transit flag
# (21) vanpool, (63) medical transportation service, excluded
# (27) paratransit, (62) employer-provided shuttle/bus, (102) other bus/shuttle vanpool included
# need to look into (102) other bus/shuttle vanpool?  Should there be any of these roll-up 100+ modes?
flag = pd.Series(index=trip.index, data=False)
for c in ['mode_1','mode_2','mode_3','mode_4']:
    flag = flag | trip[c].isin([23,24,25,26,27,28,30,41,42,53,55,61,62,67,68,78,102,105])
    
# include/exclude (10) school bus?
for c in ['mode_type']:
    flag = flag | trip[c].isin([10,11,12,13])
    
# 9 and 10 are "dropped off".  Possibly remove these.
subflag = False
for c in ['transit_access', 'transit_egress']:
    subflag = subflag | trip[c].isin([995])
    
missing_trip_transit_accegr = flag & subflag

flag = False
for c in ['o_purpose','d_purpose','o_purpose_category','d_purpose_category']:
    flag = flag | trip[c].isin([-1,995]) | pd.isnull(trip[c])
missing_trip_purpose_flag = flag

#missing_trip_mode_flag = trip['mode_type'].isin([-1,995]) | trip['mode_1'].isin([-1,995]) | pd.isnull(trip['mode_type']) | pd.isnull(trip['mode_1'])
# LENIENT CRITERIA `missing_trip_mode_flag` uses `mode_type` only
missing_trip_mode_flag = trip['mode_type'].isin([-1,995]) | pd.isnull(trip['mode_type'])
missing_trip_any_flag = missing_trip_mode_flag | missing_trip_driver_flag | missing_trip_purpose_flag | missing_trip_transit_accegr

# Flag the original
trip['sfcta_is_complete'] = (~missing_trip_any_flag)*1
s23.trip = pd.merge(s23.trip, trip[['trip_id','sfcta_is_complete']], on='trip_id', how='left')

In [13]:
mp = {0: 'incomplete',
      1: 'complete',
      995: 'missing'}

In [14]:
row, col = 0,0
sheet_name = 'Trip'
for name, flag in [('Complete User-Reported Mode', missing_trip_mode_flag),
                   ('Complete `mode_type`', trip['mode_type'].isin([-1,995])),
                   ('Complete trip purpose', missing_trip_purpose_flag),
                   ('Complete `driver` for drive trips with 2+ driving-age persons', missing_trip_driver_flag),
                   ('Complete transit access or egress mode', missing_trip_transit_accegr),
                   ('Complete trip survey all criteria', missing_trip_any_flag)]:
    
    df = trip.pivot_table(index='trip_survey_complete', columns=~flag*1, values='trip_id', aggfunc='size')
    df.index.name=None
    df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
              columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
              inplace=True)
    row, col = df_to_excel(writer,
                           sheet_name=sheet_name,
                           table_name=name,
                           df=df,
                           startrow=row, startcol=col,
                           offset_rows=True)

## Day Completeness
1. `missing_trip_basic_flag`: missing_trip `begin_day` or `end_day`
2. `missing_trip_telecommute_flag`: missing_trip `telecommute_time` for persons who are employed.
3. `missing_trip_school_flag`: missing_trip `school_daily` for students
4. `missing_trip_delivery_flag`: missing_trip any `delivery_X` for rMove users or non-rMove `person_num`==1
5. `missing_trip_no_travel_flag`: missing_trip any `no_travel_X` for days with 0 trips
6. `made_travel_no_trips_flag`: `made_travel`==1 & `num_trips`==0
7. `day_missing_trip_any_flag`: Any of the above

In [15]:
core_day_needs = ['begin_day', 'end_day', 'telecommute_time', 'school_daily',
                  'delivery_2', 'delivery_3', 'delivery_4',
                  'delivery_5', 'delivery_6', 'delivery_7', 'delivery_8', 'delivery_9',
                  'delivery_996', 'made_travel', 'no_travel_1', 'no_travel_2',
                  'no_travel_3', 'no_travel_4', 'no_travel_5', 'no_travel_6',
                  'no_travel_7', 'no_travel_8', 'no_travel_9', 'no_travel_11',
                  'no_travel_12', 'no_travel_99']

proxy_day_needs = ['attend_school_1', 'attend_school_2', 'attend_school_3',
                   'attend_school_998', 'attend_school_999', 'attend_school_no_1',
                   'attend_school_no_2', 'attend_school_no_3', 'attend_school_no_4',
                   'attend_school_no_5', 'attend_school_no_997', 'attend_school_no_998',
                   'attend_school_no_999','made_travel']

In [16]:
## Roll up the completeness analysis to the day.

tmp = (s23.trip
       .groupby(['hh_id','person_id','day_num'])
       .agg({'sfcta_is_complete':'sum','trip_id':'count'})
       .rename(columns={'sfcta_is_complete':'sfcta_num_trips'}))
tmp['sfcta_day_trips_complete'] = (tmp['sfcta_num_trips'].eq(tmp['trip_id'])*1)
s23.day = pd.merge(s23.day, tmp[['sfcta_num_trips','sfcta_day_trips_complete']], left_on=['hh_id','person_id','day_num'], right_index=True, how='left')

s23.day.loc[s23.day['made_travel'].eq(0) & pd.isnull(s23.day['sfcta_day_trips_complete']), 'sfcta_num_trips'] = 0
s23.day.loc[s23.day['made_travel'].eq(0) & pd.isnull(s23.day['sfcta_day_trips_complete']), 'sfcta_day_trips_complete'] = 1
s23.day.loc[pd.isnull(s23.day['sfcta_num_trips']) & s23.day['num_trips'].eq(0), 'sfcta_day_trips_complete'] = 1
s23.day.loc[pd.isnull(s23.day['sfcta_num_trips']) & s23.day['num_trips'].eq(0), 'sfcta_num_trips'] = 0
day = pd.merge(s23.person[['person_id','age','employment','is_proxy','has_proxy','student','diary_platform']],
               s23.day)

In [17]:
## Create day flags
flag = False
for c in ['begin_day','end_day']:
    flag = flag | day[c].eq(995) | pd.isnull(day[c])
missing_day_basic_flag = flag

flag=False
for c in ['telecommute_time']:
    flag = flag | day[c].eq(995) | pd.isnull(day[c])
missing_day_telecommute_flag = day['employment'].isin([1,2,3,7,8]) & flag

flag = False
for c in ['school_daily']:
    flag = flag | ((day['age'].le(3) & day['student'].isin([0,1])) & ((day[c].eq(995)) | (pd.isnull(day[c]))))
missing_day_school_flag = flag

flag = False
count = 0
for c in ['delivery_2','delivery_3','delivery_4','delivery_5','delivery_6','delivery_7','delivery_8','delivery_9']:
    flag = flag | day[c].eq(995) | pd.isnull(day[c])
    count = count + (day[c].eq(1) * 1)
missing_day_delivery_flag = (day['diary_platform'].eq('rmove') | 
                         (day['diary_platform'].ne('rmove') & day['person_num'].eq(1))) & (flag | (count.eq(0) & day['delivery_996'].eq(995)))

flag = False
for c in ['no_travel_1','no_travel_2','no_travel_3','no_travel_4','no_travel_5','no_travel_6','no_travel_7',
          'no_travel_8','no_travel_9','no_travel_11','no_travel_12','no_travel_99']:
    flag = flag | (day['num_trips'].eq(0) & (day[c].eq(995) | pd.isnull(day[c])))
missing_day_no_travel_flag = flag

made_travel_no_trips_flag = day['made_travel'].eq(1) & day['num_trips'].eq(0)

missing_day_any_flag = missing_day_basic_flag | missing_day_school_flag | missing_day_delivery_flag | missing_day_no_travel_flag | made_travel_no_trips_flag

day['sfcta_day_survey_complete'] = (~missing_day_any_flag)*1
day['sfcta_day_complete'] = (day['sfcta_day_survey_complete'].eq(1) & day['sfcta_day_trips_complete'].eq(1))*1

# flag the original
s23.day = pd.merge(s23.day, day[['day_id','sfcta_day_survey_complete','sfcta_day_trips_complete','sfcta_day_complete']],
                   on='day_id', how='left')

In [18]:
row, col = 0,0
sheet_name = 'Day'

for name, flag in [('Complete Basic Day Attributes', missing_day_basic_flag),
                   ('Complete `telecommute_time`', missing_day_telecommute_flag),
                   ('Complete `school`', missing_day_school_flag),
                   ('Complete delivery', missing_day_delivery_flag),
                   ('Complete no-travel', missing_day_no_travel_flag),
                   ('Made travel, but 0 trips', made_travel_no_trips_flag),
                   ('Complete day survey all criteria', missing_day_any_flag),
                   ]:
    df = day.pivot_table(index='is_complete', columns=~flag*1, values='day_id', aggfunc='count')
    df.index.name=None
    df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
              columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
              inplace=True)
    row, col = df_to_excel(writer,
                           table_name=name,
                           sheet_name=sheet_name,
                           df=df,
                           startrow=row, startcol=col,
                           offset_rows=True)

df = day.pivot_table(index='is_complete', columns='sfcta_day_trips_complete', values='day_id', aggfunc='count')
df.index.name=None
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
          inplace=True)
row, col = df_to_excel(writer,
                       table_name='All Day Trips Complete',
                       sheet_name=sheet_name,
                       df=df,
                       startrow=row, startcol=col,
                       offset_rows=True)

df = day.pivot_table(index='is_complete', columns='sfcta_day_complete', values='day_id', aggfunc='count')
df.index.name=None
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
          inplace=True)
row, col = df_to_excel(writer,
                       table_name='Day Complete Final',
                       sheet_name=sheet_name,
                       df=df,
                       startrow=row, startcol=col,
                       offset_rows=True)

## Person Completeness

In [19]:
# roll up to person level
s23.person = pd.merge(s23.person,
                      day.groupby('person_id', as_index=False)
                         .agg({'sfcta_day_complete':'sum'})
                         .rename(columns={'sfcta_day_complete':'sfcta_num_days_complete'}))
person = s23.person

In [20]:
p16 = person['age'].ge(3)
p18 = person['age'].ge(4)

flag = False
for c in ['ethnicity_1','ethnicity_2','ethnicity_3','ethnicity_4','ethnicity_997','ethnicity_999']:
    flag = flag | (p18 & (person[c].eq(995) | pd.isnull(person[c])))

missing_person_ethnicity_flag=flag

flag = False
for c in ['race_1','race_2','race_3','race_4','race_5','race_997','race_999']:
    flag = flag | (p18 & (person[c].eq(995) | pd.isnull(person[c])))
missing_person_race_flag=flag

flag = False
#for c in ['gender','age','relationship','student','disability','second_home']:
# LENIENT CRITERIA `missing_person_basic_flag` excludes `disability`
for c in ['gender','age','relationship','second_home']:
    flag = flag | person[c].eq(995) | pd.isnull(person[c])
for c in ['student','employment']:
    flag = flag | (p16 & (person[c].eq(995) | pd.isnull(person[c])))
for c in ['education']:
    flag = flag | (p18 & (person[c].eq(995) | pd.isnull(person[c])))
missing_person_basic_flag = flag
    
# LENIENT CRITERIA remove `missing_person_clipper_flag`
#flag = person['clipper_card'].eq(995)
#for c in ['clipper_card_use_1','clipper_card_use_2']:
#    flag = flag | (~flag & (person[c].eq(995) | pd.isnull(person[c])))
#missing_person_clipper_flag = flag

#flag = False
#for c in ['school_attend','school_type','school_freq','remote_class_freq']:
#    flag = flag | person[c].eq(995) | pd.isnull(person[c])

#missing_person_any_flag = missing_person_ethnicity_flag | missing_person_race_flag | missing_person_basic_flag | missing_person_clipper_flag
# LENIENT CRITERIA remove `missing_person_clipper_flag`
missing_person_any_flag = missing_person_ethnicity_flag | missing_person_race_flag | missing_person_basic_flag

In [21]:
person['sfcta_person_days_complete'] = ((person['diary_platform'].isin(['browser','call']) & person['sfcta_num_days_complete'].ge(1)) |
                                 (person['diary_platform'].eq('rmove') & person['sfcta_num_days_complete'].ge(5))) * 1
person['sfcta_person_survey_complete'] = ~missing_person_any_flag*1
person['sfcta_person_complete'] = (~missing_person_any_flag & person['sfcta_person_days_complete'].eq(1))*1

# don't need to copy back because person is an alias of s23.person

In [22]:
row, col = 0,0
sheet_name = 'Person'

for name, flag in [('Complete basic person attributes', missing_person_basic_flag),
                   ('Complete ethnicity', missing_person_ethnicity_flag),
                   ('Complete race', missing_person_race_flag),
                   ('Person Survey Complete all criteria', missing_person_any_flag),
                   ]:
    df = person.pivot_table(index='is_complete', columns=~flag*1, values='person_id', aggfunc='count')
    df.index.name=None
    df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
              columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
              inplace=True)
    row, col = df_to_excel(writer,
                           table_name=name,
                           sheet_name=sheet_name,
                           df=df,
                           startrow=row, startcol=col,
                           offset_rows=True)

df = person.pivot_table(index='is_complete', columns='sfcta_person_days_complete', values='person_id', aggfunc='count')
df.index.name=None
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
          inplace=True)
row, col = df_to_excel(writer,
                       table_name='All Person Days Complete',
                       sheet_name=sheet_name,
                       df=df,
                       startrow=row, startcol=col,
                       offset_rows=True)

df = person.pivot_table(index='is_complete', columns='sfcta_person_complete', values='person_id', aggfunc='count')
df.index.name=None
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
          inplace=True)
row, col = df_to_excel(writer,
                       table_name='Person Complete Final',
                       sheet_name=sheet_name,
                       df=df,
                       startrow=row, startcol=col,
                       offset_rows=True)

## Household Completeness

In [23]:
tmp = (s23.person
       .groupby('hh_id', as_index=False)
       .agg({'sfcta_person_complete':'sum', 'person_id':'count'})
       .rename(columns={'sfcta_person_complete':'sfcta_num_persons_complete',
                        'person_id':'persons'})
       )
tmp['sfcta_hh_persons_complete'] = (tmp['sfcta_num_persons_complete'].eq(tmp['persons']))*1
s23.hh = pd.merge(s23.hh, tmp[['hh_id','sfcta_num_persons_complete','sfcta_hh_persons_complete']],
                  on='hh_id', how='left')

In [24]:
hh= s23.hh

In [25]:
flag = False
for c in ['income_detailed','income_broad','num_workers','num_students','num_vehicles','num_bicycles']:
    flag = flag | hh[c].eq(995) | pd.isnull(hh[c])
    
flag = flag | (hh['income_detailed'].isin([995,999]) & hh['income_followup'].eq(995))
missing_hh_basic_flag = flag

In [26]:
hh['sfcta_hh_survey_complete'] = ~missing_hh_basic_flag*1
hh['sfcta_hh_complete'] = (hh['sfcta_hh_survey_complete'].eq(1) & hh['sfcta_hh_persons_complete'].eq(1))*1

In [27]:
hh.pivot_table(index='is_complete', columns=(hh['income_detailed'].isin([995,999]) & hh['income_followup'].eq(995))*1, values='hh_id', aggfunc='count')

,0,1
is_complete,,
1,8829,14


In [28]:
row, col = 0,0
sheet_name = 'Household'

for name, flag in [('Complete basic household attributes', missing_hh_basic_flag),
                   ]:
    df = hh.pivot_table(index='is_complete', columns=~flag*1, values='hh_id', aggfunc='count')
    df.index.name=None
    df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
              columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
              inplace=True)
    row, col = df_to_excel(writer,
                           table_name=name,
                           sheet_name=sheet_name,
                           df=df,
                           startrow=row, startcol=col,
                           offset_rows=True)

df = hh.pivot_table(index='is_complete', columns='sfcta_hh_persons_complete', values='hh_id', aggfunc='count')
df.index.name=None
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
          inplace=True)
row, col = df_to_excel(writer,
                       table_name='All Household Persons Complete',
                       sheet_name=sheet_name,
                       df=df,
                       startrow=row, startcol=col,
                       offset_rows=True)

df = hh.pivot_table(index='is_complete', columns='sfcta_hh_complete', values='hh_id', aggfunc='count')
df.index.name=None
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns},
          inplace=True)
row, col = df_to_excel(writer,
                       table_name='Household Complete Final',
                       sheet_name=sheet_name,
                       df=df,
                       startrow=row, startcol=col,
                       offset_rows=True)

In [29]:
row, col = 0,0
sheet_name='Summary'

df = s23.trip.pivot_table(index='trip_survey_complete', columns='sfcta_is_complete', values='trip_id', aggfunc='count')
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns}, inplace=True)
row, col = df_to_excel(writer,
            sheet_name=sheet_name,
            table_name='Trips - RSG Complete vs SFCTA Complete',
            df=df,
            startrow=row, startcol=col,
            offset_rows=True)

df = s23.day.pivot_table(index='is_complete', columns='sfcta_day_complete', values='day_id', aggfunc='count')
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns}, inplace=True)
row, col = df_to_excel(writer,
            sheet_name=sheet_name,
            table_name='Day - RSG Complete vs SFCTA Complete',
            df=df,
            startrow=row, startcol=col,
            offset_rows=True)

df = s23.person.pivot_table(index='is_complete', columns='sfcta_person_complete', values='person_id', aggfunc='count')
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns}, inplace=True)
row, col = df_to_excel(writer,
            sheet_name=sheet_name,
            table_name='Person - RSG Complete vs SFCTA Complete',
            df=df,
            startrow=row, startcol=col,
            offset_rows=True)

df = s23.hh.pivot_table(index='is_complete', columns='sfcta_hh_complete', values='hh_id', aggfunc='count')
df.rename(index={i: 'RSG {}'.format(mp[i]) for i in df.index},
          columns={c: 'SFCTA {}'.format(mp[c]) for c in df.columns}, inplace=True)
row, col = df_to_excel(writer,
            sheet_name=sheet_name,
            table_name='Household - RSG Complete vs SFCTA Complete',
            df=df,
            startrow=row, startcol=col,
            offset_rows=True)

In [30]:
writer.close()